In [29]:
from pathlib import Path

import tvm
from tvm import te
from tvm.contrib import graph_executor as runtime
from tvm import relay, transform

import onnx

import imageio
import matplotlib.pyplot as plt
import logging

import numpy as np
from PIL import Image

In [9]:
model_file = Path.cwd().parent / 'models' / 'yunet' / 'yunet.onnx'
if not model_file.exists():
    logging.error(f'Model file {model_file} not found')

In [11]:
onnx_model = onnx.load(model_file)

In [28]:
target = "llvm"

input_tensor = "input"
shape_dict = {input_tensor: (1, 3, 320, 240)}
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

target = "llvm"
with transform.PassContext(opt_level=3):
    lib=relay.build(mod, target, params=params)

In [33]:
img = imageio.imread(Path.cwd().parent / 'assets' / 'pexels-yaroslav-chaadaev-9303791.jpg')
img.shape

(5184, 3456, 3)

In [35]:
pil_image = Image.fromarray(img)
img = np.asarray(pil_image.resize((320, 240), Image.BILINEAR))

In [36]:
img = np.transpose(img, (2, 0, 1))
img.shape

(3, 240, 320)

In [44]:
img = img / 255.0
img = img.astype(np.float32)

In [45]:
image_data = img[np.newaxis]
image_data.shape

(1, 3, 240, 320)

In [46]:
# Create a runtime executor module
module = runtime.GraphModule(lib["default"](tvm.cpu()))

# Feed input data
module.set_input(input_tensor, tvm.nd.array(image_data))

# Run
module.run()

In [47]:
confidence = module.get_output(0).numpy()
location = module.get_output(1).numpy()
iou = module.get_output(2).numpy()

In [64]:
location.shape

(4385, 2)